In [3]:
import numpy as np
import os
from pathlib import Path
import pandas as pd
import numpy as np
import orjson
import zipfile
import zstandard as zstd
import shutil
from pymatgen.core import Structure
import shutil as sh
import pathlib as pl
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split
import time
import sys
sys.path.append("../../")
from utils.experiment_tracking import track_metrics, log_mean_std_based_on_test_metrics
from utils.save_and_load import save_to_json, load_from_json


In [4]:
# reformat json into .csv + .cifs inside ./data/regression/
total_ds = load_from_json("../../data/final_data/dataset_of_max_abs_shg.json")
try:
    ! rm -r ./data/regression/
except:
    pass
! mkdir -p ./data/regression
# ! cp ./data/sample-regression/atom_init.json ./data/regression
names, labels = [], []
for k, v in list(total_ds.items()):
    structure: Structure = Structure.from_dict(v["structure"])
    structure.to("./data/regression/" + k + ".cif", "cif")
    names.append(k+'.cif')
    labels.append(v["shg"])
pd.DataFrame.from_dict(dict(names=names, labels=labels)).to_csv(
    "./data/regression/id_prop.csv", header=False, index=False, sep=","
)

In [5]:
import time
from jarvis.db.jsonutils import loadjson
from alignn.train import train_dgl, ALIGNN
from alignn.models.alignn import ALIGNNConfig
import random

# from alignn.scripts.train_folder import train_for_folder
import torch

import csv
import os
from jarvis.core.atoms import Atoms

# from jarvis.core.graphs import Graph
# from alignn.models.alignn import ALIGNN
from alignn.data import get_train_val_loaders
from alignn.train import train_dgl
from alignn.config import TrainingConfig
from jarvis.db.jsonutils import loadjson
import argparse
from aim import Run
import sys


def train_for_folder(
    run: Run,
    root_dir="./data/regression/",
    config_name="./pre-trained/jv_mbj_bandgap_alignn/config.json",
    keep_data_order=False,
    classification_threshold=None,
    output_dir="/home/danya/mof/models_cmp/alignn/trained/",
    model=None,
):
    restart_model_path = "./pre-trained/jv_mbj_bandgap_alignn/checkpoint_300.pt"
    rest_config = loadjson(config_name)
    rest_config["output_dir"] = output_dir

    """Train for a folder."""
    # config_dat=os.path.join(root_dir,config_name)
    id_prop_dat = os.path.join(root_dir, "id_prop.csv")
    config = loadjson(config_name)
    if type(config) is dict:
        try:
            config = TrainingConfig(**config)
        except Exception as exp:
            print("Check", exp)

    config.keep_data_order = keep_data_order
    if classification_threshold is not None:
        config.classification_threshold = float(classification_threshold)
    if output_dir is not None:
        config.output_dir = output_dir
    with open(id_prop_dat, "r") as f:
        reader = csv.reader(f)
        data = [row for row in reader]

    dataset = []
    n_outputs = []
    multioutput = False
    lists_length_equal = True
    for i in data:
        info = {}
        cif_name = i[0]
        cif_path = os.path.join(root_dir, cif_name)

        print(cif_name)
        atoms = Atoms.from_cif(cif_path)
        info["atoms"] = atoms.to_dict()
        info["jid"] = cif_name

        tmp = [float(j) for j in i[1:]]  # float(i[1])
        if len(tmp) == 1:
            tmp = tmp[0]
        else:
            multioutput = True
        info["target"] = tmp  # float(i[1])
        n_outputs.append(info["target"])
        dataset.append(info)
    if multioutput:
        lists_length_equal = False not in [
            len(i) == len(n_outputs[0]) for i in n_outputs
        ]

    # print ('n_outputs',n_outputs[0])
    if multioutput and classification_threshold is not None:
        raise ValueError("Classification for multi-output not implemented.")
    if multioutput and lists_length_equal:
        config.model.output_features = len(n_outputs[0])
    else:
        # TODO: Pad with NaN
        if not lists_length_equal:
            raise ValueError("Make sure the outputs are of same size.")
        else:
            config.model.output_features = 1

    # enable deterministic learning
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(True)
    random.seed(42)
    np.random.seed(42)
    %env CUBLAS_WORKSPACE_CONFIG=:4096:8

    k_folds = 10
    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)
    data_range = np.arange(0, len(dataset))

    run["hparams"] = rest_config
    json_to_log = None
    fold_partition = {}
    fold_partition_names = {}
    run.set_artifacts_uri(f"file://{os.getcwd()}/artifacts/")
    for fold, (train_val_idx, test_idx) in enumerate(kfold.split(data_range)):
        print(f"FOLD {fold}")
        model = ALIGNN(ALIGNNConfig(**rest_config["model"]))
        model.load_state_dict(
            torch.load(restart_model_path, map_location=torch.device("cuda"))["model"]
        )

        run["model_parameters_count"] = sum(p.numel() for p in model.parameters())
        run["model_trainable_parameters_count"] = sum(
            p.numel() for p in model.parameters() if p.requires_grad
        )

        train_idx, val_idx = train_test_split(
            train_val_idx, train_size=8 / 9, random_state=42
        )

        # log fold partition
        fold_partition = dict(
            fold=fold_partition.get("fold", {})
            | {
                fold: dict(
                    train_idx=train_idx.tolist(),
                    val_idx=val_idx.tolist(),
                    test_idx=test_idx.tolist(),
                )
            }
        )
        fold_partition_names = dict(
            fold=fold_partition_names.get("fold", {})
            | {
                fold: dict(
                    train_names=[dataset[x]["jid"] for x in train_idx.tolist()],
                    val_names=[dataset[x]["jid"] for x in val_idx.tolist()],
                    test_names=[dataset[x]["jid"] for x in test_idx.tolist()],
                )
            }
        )

        config.output_dir = rest_config["output_dir"] + f"trained_fold_{fold}"
        (
            train_loader,
            val_loader,
            test_loader,
            prepare_batch,
        ) = get_train_val_loaders(
            dataset_array=dataset,
            target=config.target,
            n_train=config.n_train,
            n_val=config.n_val,
            n_test=config.n_test,
            train_ratio=config.train_ratio,
            val_ratio=config.val_ratio,
            test_ratio=config.test_ratio,
            batch_size=config.batch_size,
            atom_features=config.atom_features,
            neighbor_strategy=config.neighbor_strategy,
            standardize=config.atom_features != "cgcnn",
            id_tag=config.id_tag,
            pin_memory=config.pin_memory,
            workers=config.num_workers,
            save_dataloader=config.save_dataloader,
            use_canonize=config.use_canonize,
            filename=config.filename,
            cutoff=config.cutoff,
            max_neighbors=config.max_neighbors,
            output_features=config.model.output_features,
            classification_threshold=config.classification_threshold,
            target_multiplication_factor=config.target_multiplication_factor,
            standard_scalar_and_pca=config.standard_scalar_and_pca,
            keep_data_order=config.keep_data_order,
            output_dir=config.output_dir,
            id_train=train_idx,
            id_val=val_idx,
            id_test=test_idx,
            split_seed=42,
        )

        # log dataset once for all folds
        dataset_key_target = {v["jid"]: v["target"] for v in dataset}
        if json_to_log is None:
            json_to_log = dict(
                dataset_key_target=dataset_key_target,
                fold_partition=fold_partition,
                fold_partition_names=fold_partition_names,
                crystal_predictions_log_for_all_folds={},
            )
        else:
            json_to_log['fold_partition']=fold_partition
            json_to_log['fold_partition_names']=fold_partition_names
        train_dgl(
            run,
            config,
            model,
            jsons_to_log=json_to_log,
            train_val_test_loaders=[
                train_loader,
                val_loader,
                test_loader,
                prepare_batch,
            ],
            fold=fold,
        )
    log_mean_std_based_on_test_metrics(run)
    run.close()


/home/danya/miniconda3/envs/alignn/lib/python3.10/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
/home/danya/miniconda3/envs/alignn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# # Initialize a new run
# ds_name = 'dataset_of_max_abs_shg'
# run = Run(
#     experiment=f"ALIGNN FT from jv_mbj_bandgap_alignn on {ds_name}",
#     log_system_params=True,
# )

# # Log run parameters
# # run["hparams"] = vars(args)
# abs_path = os.path.abspath(f"trained/{ds_name}/")

# train_for_folder(run,output_dir=abs_path)
# run.close()

In [ ]:
ds_name = "base_dataset_of_eff_shg"
total_ds = load_from_json(f"../../data/final_data/{ds_name}.json")
try:
    ! rm -r ./data/regression/
except:
    pass
! mkdir -p ./data/regression
# ! cp ./data/sample-regression/atom_init.json ./data/regression
names, labels = [], []
for k, v in list(total_ds.items()):
    structure: Structure = Structure.from_dict(v["structure"])
    structure.to_file("./data/regression/" + k + ".cif", "cif")
    names.append(k+'.cif')
    labels.append(v["shg"])
pd.DataFrame.from_dict(dict(names=names, labels=labels)).to_csv(
    "./data/regression/id_prop.csv", header=False, index=False, sep=","
)

/home/danya/miniconda3/envs/alignn/lib/python3.10/site-packages/pymatgen/core/structure.py:2943: UserWarning: Site labels are not unique, which is not compliant with the CIF spec (https://www.iucr.org/__data/iucr/cifdic_html/1/cif_core.dic/Iatom_site_label.html):`['Co1', 'Co1', 'Co1', 'Co1', 'Co2', 'Co2', 'Co2', 'Co2', 'Ge1', 'Ge1', 'Ge1', 'Ge1', 'Ge2', 'Ge2', 'Ge2', 'Ge2', 'C1', 'C1', 'C1', 'C1', 'C2', 'C2', 'C2', 'C2', 'C3', 'C3', 'C3', 'C3', 'C4', 'C4', 'C4', 'C4', 'C5', 'C5', 'C5', 'C5', 'C6', 'C6', 'C6', 'C6', 'C7', 'C7', 'C7', 'C7', 'C8', 'C8', 'C8', 'C8', 'Cl1', 'Cl1', 'Cl1', 'Cl1', 'Cl2', 'Cl2', 'Cl2', 'Cl2', 'Cl3', 'Cl3', 'Cl3', 'Cl3', 'Cl4', 'Cl4', 'Cl4', 'Cl4', 'Cl5', 'Cl5', 'Cl5', 'Cl5', 'Cl6', 'Cl6', 'Cl6', 'Cl6', 'O1', 'O1', 'O1', 'O1', 'O2', 'O2', 'O2', 'O2', 'O3', 'O3', 'O3', 'O3', 'O4', 'O4', 'O4', 'O4', 'O5', 'O5', 'O5', 'O5', 'O6', 'O6', 'O6', 'O6', 'O7', 'O7', 'O7', 'O7', 'O8', 'O8', 'O8', 'O8']`.
  writer: Any = CifWriter(self, **kwargs)
/home/danya/miniconda3/envs

In [8]:
run = Run(
    experiment=f"ALIGNN FT from jv_mbj_bandgap_alignn on {ds_name}",
    log_system_params=True,
)

abs_path = os.path.abspath(f"trained/{ds_name}/")

train_for_folder(run, output_dir=abs_path)
run.close()

icsd-150328.cif
icsd-60080.cif
icsd-28106.cif
icsd-9290.cif
icsd-262538.cif
icsd-92822.cif
icsd-97512.cif
icsd-194046.cif
icsd-426537.cif
icsd-262502.cif
icsd-39459.cif
icsd-250323.cif
icsd-23240.cif
icsd-93013.cif
icsd-246037.cif
icsd-423429.cif
icsd-99793.cif
icsd-420286.cif
icsd-20173.cif
icsd-20155.cif
icsd-423435.cif
icsd-418166.cif
icsd-170949.cif
icsd-75594.cif
icsd-281322.cif
icsd-97675.cif
icsd-248201.cif
icsd-380488.cif
icsd-18001.cif
icsd-425177.cif
icsd-5244.cif
icsd-184715.cif
icsd-248147.cif
icsd-425176.cif
icsd-100261.cif
icsd-93014.cif
icsd-87519.cif
icsd-409640.cif
icsd-250215.cif
icsd-251537.cif
icsd-250553.cif
icsd-404439.cif
icsd-171422.cif
icsd-251863.cif
icsd-10443.cif
icsd-260114.cif
icsd-261333.cif
icsd-34700.cif
icsd-172383.cif
icsd-291596.cif
icsd-97761.cif
icsd-48025.cif
icsd-248205.cif
icsd-172402.cif
icsd-34580.cif
icsd-423143.cif
icsd-91540.cif
icsd-27647.cif
icsd-262542.cif
icsd-250666.cif
icsd-250446.cif
icsd-180730.cif
icsd-260879.cif
icsd-185040.cif
ic

/tmp/ipykernel_4146912/1510944958.py:116: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(restart_model_path, map_location=torch.device("cuda"))["model"]


MAX val: 163.39150002022944
MIN val: 0.0
MAD: 9.974615470229839
Baseline MAE: 9.062799535895595
data range 163.39150002022944 0.0


100%|██████████| 778/778 [00:10<00:00, 76.94it/s] 


Setting it to max atomic number available here, 103
Setting it to max atomic number available here, 103
Setting it to max atomic number available here, 103
building line graphs


...

Val_MAE: 3.9401
Train_MAE: 1.6048


Val_MAE: 3.9970
Train_MAE: 1.5826


Val_MAE: 4.1919
Train_MAE: 1.7855


Val_MAE: 4.1451
Train_MAE: 1.6403


Val_MAE: 4.0339
Train_MAE: 1.6166
Test MAE: 7.062184606487959
